# Recomendação - Recursos Semelhantes
Por meio de algoritmo de **Filtragem Baseada em Conteúdo**, recomendar recursos educacionais similares, baseado nas características desses recursos

In [1]:
import pandas as pd

kt1 = pd.read_csv(r'D:\Doutorado\USP\SCC5966 - Recomendaçao\Projeto\dataset\compacto\kt1.txt',sep='\t')
kt2 = pd.read_csv(r'D:\Doutorado\USP\SCC5966 - Recomendaçao\Projeto\dataset\compacto\kt2.txt',sep='\t')
kt3 = pd.read_csv(r'D:\Doutorado\USP\SCC5966 - Recomendaçao\Projeto\dataset\compacto\kt3.txt',sep='\t')
kt4 = pd.read_csv(r'D:\Doutorado\USP\SCC5966 - Recomendaçao\Projeto\dataset\compacto\kt4.txt',sep='\t')

In [3]:
kt1.head()

,Unnamed: 0,timestamp,solving_id,question_id,user_answer,elapsed_time,codigo_arquivo,usuario
0,0,1546437487910,1,q5956,c,50000,u5.csv,u5
1,1,1546437515076,2,q1114,b,23000,u5.csv,u5
2,2,1546437532942,3,q784,b,16000,u5.csv,u5
3,3,1546437556636,4,q5627,a,21000,u5.csv,u5
4,4,1546437605933,5,q5055,b,47000,u5.csv,u5


In [5]:
kt2.head()

,Unnamed: 0,timestamp,action_type,item_id,source,user_answer,platform,codigo_arquivo,usuario
0,0,1567144866991,enter,b4484,diagnosis,NaN,mobile,u8.csv,u8
1,1,1567144890954,respond,q5952,diagnosis,a,mobile,u8.csv,u8
2,2,1567144897002,submit,b4484,diagnosis,NaN,mobile,u8.csv,u8
3,3,1567144897077,enter,b3176,diagnosis,NaN,mobile,u8.csv,u8
4,4,1567144934774,respond,q4644,diagnosis,b,mobile,u8.csv,u8


In [50]:
kt2['action_type'].unique()

array(['enter', 'respond', 'submit'], dtype=object)

In [7]:
kt3.head()

,Unnamed: 0,timestamp,action_type,item_id,source,user_answer,platform,codigo_arquivo,usuario
0,0,1566532547478,enter,b5298,my_note,NaN,web,u5.csv,u5
1,1,1566532582693,respond,q7242,my_note,a,web,u5.csv,u5
2,2,1566532645820,respond,q7243,my_note,a,web,u5.csv,u5
3,3,1566532707773,respond,q7244,my_note,a,web,u5.csv,u5
4,4,1566532753222,respond,q7245,my_note,c,web,u5.csv,u5


In [52]:
kt3['action_type'].unique()

array(['enter', 'respond', 'submit', 'quit'], dtype=object)

In [9]:
kt4.head()

,Unnamed: 0,timestamp,action_type,item_id,cursor_time,source,user_answer,platform,codigo_arquivo,usuario
0,0,1565651017866,pay,p130,NaN,NaN,NaN,NaN,u5.csv,u5
1,1,1566532547478,enter,b5298,NaN,my_note,NaN,web,u5.csv,u5
2,2,1566532582693,respond,q7242,NaN,my_note,a,web,u5.csv,u5
3,3,1566532645820,respond,q7243,NaN,my_note,a,web,u5.csv,u5
4,4,1566532707773,respond,q7244,NaN,my_note,a,web,u5.csv,u5


In [54]:
kt4['action_type'].unique()

array(['pay', 'enter', 'respond', 'submit', 'quit', 'erase_choice',
       'play_video', 'pause_video', 'play_audio', 'pause_audio',
       'undo_erase_choice', 'enroll_coupon', 'refund'], dtype=object)

In [11]:
lectures = pd.read_csv('D:/Doutorado/USP/SCC5966 - Recomendaçao/Projeto/dataset/EdNet-Contents/lectures.csv')
lectures.head()

,lecture_id,part,tags,video_length,deployed_at
0,l520,5,142,-1,-1
1,l592,6,142,-1,-1
2,l1259,1,222,359000,1570424729123
3,l1260,1,220,487000,1570424738105
4,l1261,1,221,441000,1570424743162


In [13]:
questions = pd.read_csv('D:/Doutorado/USP/SCC5966 - Recomendaçao/Projeto/dataset/EdNet-Contents/questions.csv')
questions.head()

,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,q1,b1,e1,b,1,1;2;179;181,1558093217098
1,q2,b2,e2,a,1,15;2;182,1558093219720
2,q3,b3,e3,b,1,14;2;179;183,1558093222784
3,q4,b4,e4,b,1,9;2;179;184,1558093225357
4,q5,b5,e5,c,1,8;2;179;181,1558093228439


In [15]:
resposta_correta = questions.loc[(questions.question_id == 'q1'),'correct_answer'][0]
resposta_correta

'b'

### Utilizando as tags dos recursos como embeddings

In [19]:
questions.head()

,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,q1,b1,e1,b,1,1;2;179;181,1558093217098
1,q2,b2,e2,a,1,15;2;182,1558093219720
2,q3,b3,e3,b,1,14;2;179;183,1558093222784
3,q4,b4,e4,b,1,9;2;179;184,1558093225357
4,q5,b5,e5,c,1,8;2;179;181,1558093228439


In [46]:
import pandas as pd

# Passo 1: Separar os valores por ponto e vírgula
questions['tags_separadas'] = questions['tags'].str.split(';')

# Passo 2: Explodir para trabalhar com cada número individualmente
tags_explodes = questions.explode('tags_separadas')

# Converter os valores para números inteiros (se necessário)
tags_explodes['tags_separadas'] = tags_explodes['tags_separadas'].astype(int)

# Passo 3: Criar a matriz one-hot encoding
tags_one_hot = pd.get_dummies(tags_explodes, columns=['tags_separadas'], prefix='', prefix_sep='')

# Passo 4: Agrupar novamente por ID e somar as colunas one-hot
questions_tags_one_hot = tags_one_hot.groupby('question_id').max()

# Removendo qualquer coluna extra (deixando apenas ID e as colunas one-hot)
questions_tags_one_hot = questions_tags_one_hot.reset_index()

print("\nDataFrame com matriz one-hot encoding:")
questions_tags_one_hot


DataFrame com matriz one-hot encoding:


,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at,-1,1,2,...,181,182,183,184,185,186,194,196,219,300
0,q1,b1,e1,b,1,1;2;179;181,1558093217098,False,True,True,...,True,False,False,False,False,False,False,False,False,False
1,q10,b10,e10,c,1,17;7;182,1558093243018,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,q100,b100,e100,c,1,22;2;181,1514558485914,False,False,True,...,True,False,False,False,False,False,False,False,False,False
3,q1000,b1000,e1000,c,2,24;33;182;183,-1,False,False,False,...,False,True,True,False,False,False,False,False,False,False
4,q10000,b7351,e7351,b,5,83,1574065074140,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,q9995,b7346,e7346,d,5,83,1568688413601,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13165,q9996,b7347,e7347,a,5,83,1574065066829,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13166,q9997,b7348,e7348,d,5,83,1574065069466,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13167,q9998,b7349,e7349,a,5,83,1568688415765,False,False,False,...,False,False,False,False,False,False,False,False,False,False
